In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/modelpb/labels.txt
/kaggle/input/modelpb/model.pb
/kaggle/input/imagefile/TestFile.PNG


In [2]:
import tensorflow as tf
from tensorflow.python.platform import gfile

In [3]:
   import tensorflow as tf

GRAPH_PB_PATH = '../input/modelpb/model.pb'

with tf.compat.v1.Session() as sess:
    print("load graph")
    with tf.io.gfile.GFile(GRAPH_PB_PATH,'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    graph_nodes=[n for n in graph_def.node]
    names = []
    for t in graph_nodes:
        names.append(t.name)
    print(names)
    
    
#adjustments because I am using Tensorflow 2
# tf.compat.v1.GraphDef()   # -> instead of tf.GraphDef()
# tf.compat.v2.io.gfile.GFile()   # -> instead of tf.gfile.GFile()
    
import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.compat.v1.Session() as sess:
    with gfile.FastGFile(GRAPH_PB_PATH, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='C:/Users/WLi/Desktop'
writer = tf.summary.create_file_writer(LOGDIR)
#writer.add_graph(sess.graph)

#train_writer.flush()


with writer.as_default():
      for step in range(100):
    # other model code would go here
        tf.summary.scalar("my_metric", 0.5, step=step)
        #writer.flush()
    

load graph
['conv0/pad_size/paddings', 'block1_0_reshape1/shape', 'block2_2_conv1/weights', 'block1_3_conv2/biases', 'block2_3_conv1/weights', 'block0_3_conv1/biases', 'block0_0_concat/axis', 'block2_1_conv2/biases', 'block0_1_slice-Slice0/size', 'block0_0_conv0/biases', 'block0_0_conv1/biases', 'block2_0_conv1/weights', 'block1_4_conv1/biases', 'block0_0_conv3/biases', 'block1_3_conv1/weights', 'block0_0_transpose/perm', 'block2_1_conv1/biases', 'block2_2_conv2/biases', 'block1_7_conv0/weights', 'block1_5_conv1/weights', 'block0_0_conv4/weights', 'block1_6_conv2/biases', 'block1_3_conv0/biases', 'block0_2_conv2/biases', 'block1_6_conv1/weights', 'block2_0_conv4/weights', 'block1_7_conv2/biases', 'block0_1_slice-Slice1/begin', 'block2_3_conv1/biases', 'block1_2_conv2/biases', 'block0_2_conv1/biases', 'block1_1_slice-Slice0/size', 'block1_2_conv0/biases', 'block2_0_conv2/weights', 'block1_2_conv1/weights', 'block2_0_conv4/biases', 'block1_0_conv1/weights', 'block0_1_conv1/weights', 'blo

In [4]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.3.1


In [5]:
#%tensorboard --logdir logs

Try out:
https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/export-model-python

In [6]:
import tensorflow as tf
import os

graph_def = tf.compat.v1.GraphDef()
labels = []

# These are set to the default names from exported models, update as needed.
filename = "../input/modelpb/model.pb"
labels_filename = "../input/modelpb/labels.txt"

# Import the TF graph
with tf.io.gfile.GFile(filename, 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

# Create a list of labels.
with open(labels_filename, 'rt') as lf:
    for l in lf:
        labels.append(l.strip())

In [7]:
#importing helperfunctions

def convert_to_opencv(image):
    # RGB -> BGR conversion is performed as well.
    image = image.convert('RGB')
    r,g,b = np.array(image).T
    opencv_image = np.array([b,g,r]).transpose()
    return opencv_image

def crop_center(img,cropx,cropy):
    h, w = img.shape[:2]
    startx = w//2-(cropx//2)
    starty = h//2-(cropy//2)
    return img[starty:starty+cropy, startx:startx+cropx]

def resize_down_to_1600_max_dim(image):
    h, w = image.shape[:2]
    if (h < 1600 and w < 1600):
        return image

    new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
    return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)

def resize_to_256_square(image):
    h, w = image.shape[:2]
    return cv2.resize(image, (256, 256), interpolation = cv2.INTER_LINEAR)

def update_orientation(image):
    exif_orientation_tag = 0x0112
    if hasattr(image, '_getexif'):
        exif = image._getexif()
        if (exif != None and exif_orientation_tag in exif):
            orientation = exif.get(exif_orientation_tag, 1)
            # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
            orientation -= 1
            if orientation >= 4:
                image = image.transpose(Image.TRANSPOSE)
            if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)
            if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

In [8]:
from PIL import Image
import numpy as np
import cv2

# Load from a file
imageFile = "../input/imagefile/TestFile.PNG"
image = Image.open(imageFile)

# Update orientation based on EXIF tags, if the file has orientation info.
image = update_orientation(image)

# Convert to OpenCV format
image = convert_to_opencv(image)

In [9]:
print(image.shape)

(213, 293, 3)


In [10]:
# If the image has either w or h greater than 1600 we resize it down respecting
# aspect ratio such that the largest dimension is 1600
image = resize_down_to_1600_max_dim(image)

In [11]:
# We next get the largest center square
h, w = image.shape[:2]
min_dim = min(w,h)
max_square_image = crop_center(image, min_dim, min_dim)

In [12]:
# Resize that square down to 256x256
augmented_image = resize_to_256_square(max_square_image)

In [13]:
# Get the input size of the model
with tf.compat.v1.Session() as sess:
    input_tensor_shape = sess.graph.get_tensor_by_name('Placeholder:0').shape.as_list()
network_input_size = input_tensor_shape[1]

# Crop the center for the specified network_input_Size
augmented_image = crop_center(augmented_image, network_input_size, network_input_size)

In [14]:
print(augmented_image.shape)

(224, 224, 3)


In [15]:
# These names are part of the model and cannot be changed.
output_layer = 'loss:0'
input_node = 'Placeholder:0'

with tf.compat.v1.Session() as sess:
    try:
        prob_tensor = sess.graph.get_tensor_by_name(output_layer)
        predictions = sess.run(prob_tensor, {input_node: [augmented_image] })
    except KeyError:
        print ("Couldn't find classification output layer: " + output_layer + ".")
        print ("Verify this a model exported from an Object Detection project.")
        exit(-1)

In [16]:
    # Print the highest probability label
    highest_probability_index = np.argmax(predictions)
    print('Classified as: ' + labels[highest_probability_index])
    print()

    # Or you can print out all of the results mapping labels to probabilities.
    label_index = 0
    for p in predictions:
        truncated_probablity = np.float64(np.round(p,8))
        print (labels[label_index], truncated_probablity)
        label_index += 1

Classified as: Negative

Negative [0.53689587 0.07309896 0.39000517]
